In [1]:
%matplotlib inline

import time
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from collections import defaultdict
from reachy_pyluos_hal.pycore import GateProtocol

data = defaultdict(lambda: defaultdict(list))

class GateHandler(GateProtocol):
    def handle_dxl_pub_data(self, register, ids, errors, values):
        for id, err, val in zip(ids, errors, values):
            data[id][register].append((err, val, time.time()))
    
    def handle_orbita_pub_data(self, orbita_id, reg_type, values):
        data[orbita_id][reg_type].append((values, time.time()))
    
    def handle_assert(self, msg):
        raise AssertionError(msg)

In [3]:
import sys

from serial import Serial
from serial.threaded import ReaderThread

if sys.platform == 'darwin':
    s = Serial('/dev/tty.usbserial-DN05NM0W', baudrate=1000000)
else:
    s = Serial('/dev/ttyUSB0', baudrate=1000000)
    s.set_low_latency_mode(True)

start = time.time()
with ReaderThread(s, GateHandler) as p:
    while time.time() - start < 60:
        p.send_keep_alive()
        time.sleep(1.0)

In [4]:
ids = data.keys()
ids

dict_keys([14, 10, 15, 11, 16, 12, 13])

In [5]:
for id in ids:
    print(data[id].keys())

dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])
dict_keys([36, 43])


In [13]:
for id in ids:
    err, pos, t = zip(*data[id][36])
    elapsed_time = t[-1] - t[0]
    print(f'Motor {id} received {len(pos)} pos update (in {elapsed_time:.1f}s)')
    dt = np.diff(t) * 1000
    print(f'\tAvg Dt {np.mean(dt):.1f}ms (Std={np.std(dt):.1f})')
    

Motor 14 received 34887 pos update (in 60.1s)
	Avg Dt 1.7ms (Std=0.6)
Motor 10 received 26169 pos update (in 60.1s)
	Avg Dt 2.3ms (Std=0.9)
Motor 15 received 34885 pos update (in 60.1s)
	Avg Dt 1.7ms (Std=0.6)
Motor 11 received 26168 pos update (in 60.1s)
	Avg Dt 2.3ms (Std=0.9)
Motor 16 received 34885 pos update (in 60.1s)
	Avg Dt 1.7ms (Std=0.6)
Motor 12 received 26168 pos update (in 60.1s)
	Avg Dt 2.3ms (Std=0.9)
Motor 13 received 26168 pos update (in 60.1s)
	Avg Dt 2.3ms (Std=0.9)
